In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-11 23:53:22,0.43945,0.0,0.0,0.0,2022-06-11,23:53:22,23
1,2022-06-11 23:54:22,0.48828,0.0,0.0,0.0,2022-06-11,23:54:22,23
2,2022-06-11 23:55:22,0.46387,0.0,0.0,0.0,2022-06-11,23:55:22,23
3,2022-06-11 23:56:22,0.46387,0.0,0.0,0.0,2022-06-11,23:56:22,23
4,2022-06-11 23:57:22,0.46387,0.0,0.0,0.0,2022-06-11,23:57:22,23


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
17199,2022-06-24 11:55:47,2022-06-24,11:55:47,11,21.99707,0.00000,0.000000,0.000000
17200,2022-06-24 11:56:47,2022-06-24,11:56:47,11,15.84473,4.36734,69.199323,0.069199
17201,2022-06-24 11:57:47,2022-06-24,11:57:47,11,15.72266,4.56225,71.730706,0.071731
17202,2022-06-24 11:58:48,2022-06-24,11:58:48,11,16.55274,6.31671,104.558858,0.104559
17203,2022-06-24 11:59:48,2022-06-24,11:59:48,11,21.92383,0.00000,0.000000,0.000000


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-11T00:00:00.000000000', '2022-06-12T00:00:00.000000000',
       '2022-06-13T00:00:00.000000000', '2022-06-14T00:00:00.000000000',
       '2022-06-15T00:00:00.000000000', '2022-06-16T00:00:00.000000000',
       '2022-06-17T00:00:00.000000000', '2022-06-18T00:00:00.000000000',
       '2022-06-19T00:00:00.000000000', '2022-06-20T00:00:00.000000000',
       '2022-06-21T00:00:00.000000000', '2022-06-22T00:00:00.000000000',
       '2022-06-23T00:00:00.000000000', '2022-06-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

In [7]:
filter_daily_values = df[(df['Date'] >= '2022-06-18') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
8169,2022-06-18,0,0.0
8170,2022-06-18,0,0.0
8171,2022-06-18,0,0.0
8172,2022-06-18,0,0.0
8173,2022-06-18,0,0.0
...,...,...,...
16481,2022-06-23,23,0.0
16482,2022-06-23,23,0.0
16483,2022-06-23,23,0.0
16484,2022-06-23,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values.tail(30)

,Date,Hour,Power (KW)
114,2022-06-22,18,0.222812
115,2022-06-22,19,0.232382
116,2022-06-22,20,0.009248
117,2022-06-22,21,0.000000
118,2022-06-22,22,0.000000
119,2022-06-22,23,0.000000
120,2022-06-23,0,0.000000
121,2022-06-23,1,0.000000
122,2022-06-23,2,0.000000
123,2022-06-23,3,0.000000


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail(30)

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
138,55.5,22.7,44,25
139,42.2,21.1,53,25
140,16.5,20.2,60,70
141,5.2,19.5,69,70
142,0.0,18.1,78,68
143,0.0,16.8,88,70
144,0.0,16.2,91,70
145,0.0,16.3,90,70
146,0.0,16.3,86,70
147,0.0,14.8,92,70


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(30)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
138,0.362591,55.5,22.7,44,25
139,0.072253,42.2,21.1,53,25
140,0.002300,16.5,20.2,60,70
141,0.000000,5.2,19.5,69,70
142,0.000000,0.0,0.0,0,0
143,0.000000,0.0,0.0,0,0
144,NaN,0.0,0.0,0,0
145,NaN,0.0,0.0,0,0
146,NaN,0.0,0.0,0,0
147,NaN,0.0,0.0,0,0


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

#### Is model well fit or not?

In [16]:
r_sq = reg.score(independent_columns, dependent_column)
r_sq

0.33035761737620206

In [17]:
reg.intercept_

0.0

In [18]:
reg.coef_

array([ 0.00738691,  0.03705704, -0.00059478, -0.00536309])

In [19]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [20]:
return_array = list(reg.predict(forcasted_data))
return_array

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.12242466518073586,
 0.30693216846321314,
 0.46541706601578414,
 0.4723867748090683,
 0.5004424316660836,
 0.49405516800908655,
 0.5775535554234421,
 0.6444815280644536,
 0.5915614204396362,
 0.6364848093852822,
 0.6802738674735225,
 0.6843879291642074,
 0.5852283948368504,
 0.4669700537297182,
 0.3270776280305149,
 0.2265229789095664,
 0.1104435627994591,
 0.0,
 0.0]

In [21]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [22]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [23]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [24]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [25]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-24,0,0.000000
1,2022-06-24,1,0.000000
2,2022-06-24,2,0.000000
3,2022-06-24,3,0.000000
4,2022-06-24,4,0.000000
5,2022-06-24,5,0.122425
6,2022-06-24,6,0.306932
7,2022-06-24,7,0.465417
8,2022-06-24,8,0.472387
9,2022-06-24,9,0.500442


## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [26]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'])
mean_sq_error

ValueError: Found input variables with inconsistent numbers of samples: [12, 24]

In [ ]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

In [ ]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'])
mean_ab_error

#### R squared

In [ ]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'])
r_squared

In [ ]:
# dependent_variable = 'Power (KW)'

In [ ]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [ ]:
# independent_variables

In [ ]:
# X = df1[independent_variables].values

In [ ]:
# y = df1[dependent_variable].values

In [ ]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# X_train

In [ ]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [ ]:
# y_pred = regressor.predict(X_test)
# y_pred

In [ ]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# r2_score(y_test, y_pred)